In [ ]:
from torch import nn
from MileStone_1_Training_Validation_data_Setup.load_data import load_data_from_ImageFolder

SOURCE_DIR = "./Dataset_Willys_2020/ORGINAL/"
dataset = load_data_from_ImageFolder(root=SOURCE_DIR)
#criterion = nn.SmoothL1Loss()


# MileStone 2


In [ ]:
import torch
valid_split = 0.15
test_split = 0.15
batch_size = 256
n_jobs = 24
n_epochs = 5
#criterion = nn.CrossEntropyLoss()
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
import numpy as np



num_train = len(dataset)
indices = list(range(num_train))
split = int(valid_split*num_train)


valid_idx = np.random.choice(indices,size=split,replace=False)
train_idx = list(set(indices)-set(valid_idx))
#test_idx = 

train_sampler= SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = DataLoader(dataset, sampler=train_sampler, batch_size=batch_size,
num_workers=n_jobs,pin_memory=True, drop_last= True)
valid_loader = DataLoader(dataset,sampler=valid_sampler, batch_size=batch_size
, num_workers=n_jobs,pin_memory = True, drop_last= True)



In [ ]:
train_temp, label_temp = next(iter(train_loader))
label_temp.shape, train_temp.shape


In [ ]:
label_temp.type(torch.long)
type(label_temp), label_temp.dtype

In [ ]:
#labels = dataset.classes
#len(labels)

In [ ]:
import matplotlib.pyplot as plt
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    #mean = np.array([0.485, 0.456, 0.406])
    #std = np.array([0.229, 0.224, 0.225])
    #inp = std * inp + mean
    #inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [ ]:
from torchvision.utils import make_grid
#inputs, classes = sample
#out = make_grid(inputs)

In [ ]:
#imshow(out, title=[labels[x] for x in classes])

In [ ]:
#labels

# MileStone 3


In [ ]:
import torch
assert torch.cuda.is_available()
from torchinfo import summary
from torchvision import models
device = torch.device('cuda:0')
torch.backends.cudnn.benchmark=True

In [ ]:
from torchvision import models
from torchinfo import summary
pretrained = models.mnasnet0_5(pretrained=True)
#summary(model,input_size=((5,3,198,198)))

In [ ]:
for param in pretrained.parameters():
    param.requires_grad = False
    

In [ ]:
summary(pretrained)

In [ ]:
# Hi, For this, there are different approaches but personally, I would create a class on top of previously
# defined model which you have weights for. Then add any other layers as another sequential to the new defined model.
# Something like this:
#pretrained = torchvision.models.alexnet(pretrained=True)
#https://discuss.pytorch.org/t/load-only-a-part-of-the-network-with-pretrained-weights/88397

import torch
assert torch.cuda.is_available()
from torchinfo import summary
from torchvision import models
device = torch.device('cuda:0')
torch.backends.cudnn.benchmark=True
from torch import nn
pretrained = models.mnasnet0_5(pretrained=True)

class MyAlexNet(nn.Module):
    def __init__(self, my_pretrained_model):
        super(MyAlexNet, self).__init__()
        self.pretrained = my_pretrained_model
        self.my_new_layers = nn.Linear(1000,out_features=18,bias=True)

    def forward(self,x):
        x = self.pretrained(x)
        x = self.my_new_layers(x)
        return x

 
mm = MyAlexNet(pretrained)
summary(mm)

In [ ]:
from MileStone_3_Create_Neural_Network.WillyNet import WillyNet
import torch
assert torch.cuda.is_available()
from torchinfo import summary
from torch.nn import functional as F

device = torch.device('cuda:0')
torch.backends.cudnn.benchmark=True
from torch import nn

model = WillyNet()
summary(model)

In [ ]:
#model = MyAlexNet(pretrained)
#num_out = model.classifier[1].out_features
summary(model,input_size=((5,3,198,198)))

In [ ]:
model.cuda(device=device)
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-4,weight_decay=1e-6)
optimizer = torch.optim.AdamW(model.parameters(),weight_decay=1e-6)
# criterion = nn.SmoothL1Loss()
#criterion = nn.CrossEntropyLoss()
#criterion = F.nll_loss()


In [ ]:
def accuracy(predictions, labels):
    classes = predictions#torch.argmax(predictions, dim=1)
    #labell = torch.argmax(labels,1)
    #print(classes.shape, labels.shape)
    return torch.mean((classes == labels).float())


# MileStone 4

In [ ]:
def F_score(logit, label, threshold=0.5, beta=2):
    prob = torch.sigmoid(logit)
    prob = prob > threshold
    label = label > threshold
    TP = (prob & label).sum().float()
    TN = ((~prob) & (~label)).sum().float()
    FP = (prob & (~label)).sum().float()
    FN = ((~prob) & label).sum().float()
    accuracy = (TP+TN)/(TP+TN+FP+FN)
    precision = torch.mean(TP / (TP + FP + 1e-12))
    recall = torch.mean(TP / (TP + FN + 1e-12))
    F2 = (1 + beta**2) * precision * recall / (beta**2 * precision + recall + 1e-12)
    return accuracy, precision, recall, F2.mean(0)

In [ ]:
#from torch import nn
train_losses, valid_losses = [], []
n_batches = len(train_loader)
running_acc = 0
running_val_acc = 0
acc, prec, recall,F2 = [],[],[],[]
val_acc, val_prec, val_recall,val_F2 = [],[],[],[]
for t in range(1, n_epochs + 1):
    # training
    t_losses = []
    model.train(True)
    for i, (train_batch, train_label) in enumerate(train_loader):
        # label need to be torch.long for nn.CrossEntropyLoss()
        #train_label = train_label.type(torch.long)
        # train_label = torch.nn.functional.one_hot(train_label,18).type(torch.FloatTensor).to(device)
        train_batch, train_label = train_batch.to(device), train_label.type(torch.long).to(device)
        # print(train_batch.shape) 
        
        optimizer.zero_grad()
        train_outputs = model(train_batch)

        # for nn.CrossEntropyLoss() preds should be feeded as is after F.log_softmax()
        preds = F.log_softmax(train_outputs,dim=1)
        #_, preds = torch.max(train_outputs,1)
        #outputs_ = torch.nn.functional.one_hot(torch.argmax(train_outputs, 1), 18).type(torch.FloatTensor).to(device)
        #label = torch.nn.functional.one_hot(label,num_classes=len(labels))
        #print(outputs_.shape, label.shape)
        
        # for nn.CrossEntropyLoss() you just add the output from F.log_softmax()
        loss = F.nll_loss(preds, train_label)#criterion(preds, train_label)
        #loss = criterion(train_outputs, train_label)
        
        
        #running_acc += accuracy(outputs_,train_label)
        #acc_train, precision_train, recall_train, F2_train = F_score(outputs_,train_label)
        #acc.append(float(acc_train)); prec.append(float(precision_train)); recall.append(float(recall_train)); F2.append(float(F2_train))

        t_losses.append(loss.item())
        loss.backward()
        optimizer.step()
    train_losses.append(t_losses)
    #running_acc /=len(train_loader)

    # validation
    v_losses = []
    model.train(False)
    with torch.set_grad_enabled(False):
        for val_batch, val_label in valid_loader:
            # for nn.CrossEntropyLoss()
            val_label = val_label.type(torch.long)
            # val_label = torch.nn.functional.one_hot(val_label,18).type(torch.FloatTensor)
            val_batch, val_label = val_batch.to(device), val_label.to(device)            
            
            # also for nn.CrossEntropyLoss()
            val_outputs = model(val_batch)
            preds = F.log_softmax(val_outputs,dim=1)
            # _, val_preds = torch.max(val_outputs,1)            
            # val_outputs_ = torch.nn.functional.one_hot(torch.argmax(val_outputs, 1), 18)
            val_loss = F.nll_loss(preds,val_label)#criterion(preds, val_label) 
            # val_loss = criterion(val_outputs, val_label)
            
            
            v_losses.append(val_loss.item())
            #running_val_acc +=accuracy(val_outputs_,val_label)
            #acc_, precision_, recall_, F2_ = F_score(val_outputs_,val_label)
            #val_acc.append(float(acc_)); val_prec.append(float(precision_)); val_recall.append(float(recall_)); val_F2.append(float(F2_))
        valid_losses.append(v_losses)
        #running_val_acc/=len(valid_loader)

    if not np.all(np.isfinite(t_losses)):
        raise RuntimeError(
            'NaN or Inf in training loss, cannot recover. Exiting.')
    log = f'Epoch: {t} - Training Loss: {np.mean(t_losses):.2e}, Validation Loss: {np.mean(v_losses):.2e}'
    print(log)
    #log_acc = f'train_acc: {running_acc:.6f} - val_acc:   {running_val_acc:.6f}'
    #print(log_acc)
#    log_acc = f'F2_acc:    {float(acc[-1]):.6f} - F2_val: {float(val_acc[-1]):.6f}'
 #   print(log_acc)


In [ ]:
print(n_batches)

In [ ]:
sum(recall)/len(train_loader),sum(F2)/len(train_loader)

In [ ]:
Epoch: 1 - Training Loss: 1.26e-01, Validation Loss: 4.74e-02
train_acc: 0.900206 - val_acc: 0.905265
Epoch: 2 - Training Loss: 4.65e-02, Validation Loss: 4.20e-02
train_acc: 0.937633 - val_acc: 1.066907
Epoch: 3 - Training Loss: 3.40e-02, Validation Loss: 4.25e-02
train_acc: 0.944765 - val_acc: 1.092722
Epoch: 4 - Training Loss: 2.99e-02, Validation Loss: 4.10e-02
train_acc: 0.948942 - val_acc: 1.100060
Epoch: 5 - Training Loss: 2.80e-02, Validation Loss: 4.06e-02
train_acc: 0.950584 - val_acc: 1.102230

In [ ]:
torch.save(model.state_dict(),"./MyModel")

In [ ]:

model = MyAlexNet(pretrained)
model.load_state_dict(torch.load("./MyModel"))
summary(model)

In [ ]:
https://pytorch.org/tutorials/beginner/saving_loading_models.html#what-is-a-state-dict